# COSMOSIS VALIDATION

**Description**: This notebook serves as a validation of the `CosmoSIS` pipeline. Here we shall test both methods that we have implemented:

* **CASE 1**: `CosmoSIS` is run with `Cobaya` under-the-hood, with the Boltzmann solver being run through `Cobaya`. 

* **CASE 2**: `CosmoSIS` is called completely independent from `Cobaya`, even for the Boltzmann solver calculations.

The difference between the two cases are explained in more detail in the README.md file within this folder. For each case, we will provide a fixed set of parameters and calculate the final $\chi2$ value of the likelihood, comparing it with the value obtained with the fiducial CLOE/Cobaya pipeline. 

In [ ]:
# General python imports

import numpy as np
import matplotlib.pyplot as plt
import time 
import os, sys

In [ ]:
# If you have clone the repository and open this notebook,
# this notebook should be in likelihood-implementation/notebooks
# Let's set the working directory to be likelihood-implementation

likelihood_path = os.path.realpath(os.path.join(os.getcwd(),'..'))
sys.path.insert(0, likelihood_path)
print('Setting as working directory: ', likelihood_path)

In [ ]:
# Matplotlib params set-up

%matplotlib inline
plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('font',size=25)
plt.rc('axes', titlesize=26)
plt.rc('axes', labelsize=25)
plt.rc('lines', linewidth=2)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')

`Cobaya` needs some modules to run: i.e: CAMB, CLASS, Polychord, other likelihood codes, and data (Planck 18, DES...)
If you have installed the CLOE as in the README instructions, you need to activate the conda environment `cloe` to run. This conda environment has CAMB installed, so you won't need to worry about anything else

Alternatively, if you have already installed Cobaya and other Cosmological codes such as CAMB, CLASS, Polychord, or Planck 18 you have 2 options:

* **(1)**:  point out where each of them is installed with the flag 'path' in the dictionary of cell 5. See the comment in the cell 5 corresponding to the 'theory' key of the 'info' dictionary


* **(2)**: if you installed the Cosmological codes as Cobaya automatic installation suggests  (https://cobaya.readthedocs.io/en/latest/installation_cosmo.html) you need to point out the path to your modules
as in the variable `modules_path` below:

In [ ]:
# ATTENTION: write down the path to your COBAYA modules if you want to follow option (2) above.
# Otherwise skip this cell
# modules_path': "YOUR_PATH_HERE"

## Create `Cobaya` benchmark

Here we first run the fiducial CLOE/Cobaya pipeline to obtain the $\chi^2$ value of one point. We do this by creating the `model` wrapper of `Cobaya` and making an evaluation of a single point (following the method of Case 2.1 in `DEMO.ipynb`)

In [ ]:
# Import external loglike from CLOE; that is, the class EuclidLikelihood within cobaya_interface.py

from cloe.cobaya_interface import EuclidLikelihood

In [ ]:
# We are running the Euclid-Likelihood as an external likelihood class for Cobaya
# Cobaya needs a dictionary or yaml file as input to start running
info = {
    #'params': Cobaya's protected key of the input dictionary. 
    # Includes the parameters that the user would like to sample over:
'params': {
        'ombh2': 0.022, #Omega density of baryons times the reduced Hubble parameter squared
        'omch2': 0.1202, #Omega density of cold dark matter times the reduced Hubble parameter squared
        'H0': 67.0, #Hubble parameter evaluated today (z=0) in km/s/Mpc
        'tau': 0.0925, #optical depth
        'mnu': 0.06, #  sum of the mass of neutrinos in eV
        'nnu': 3.046, #N_eff of relativistic species 
        'As': 2.12605e-9, #Amplitude of the primordial scalar power spectrum
        'ns': 0.96, # primordial power spectrum tilt 
        'w': -1.0, #Dark energy fluid model
        'wa': 0.0, #Dark energy fluid model
        'omk': 0.0, #curvature density
        'omegam': None, #DERIVED parameter: Omega matter density
        'omegab': None, #DERIVED parameter: Omega baryon density
        'omeganu': None, #DERIVED parameter: Omega neutrino density
        'omnuh2': None, #DERIVED parameter: Omega neutrino density times de reduced Hubble parameter squared
        'omegac': None, #DERIVED parameter: Omega cold dark matter density
        'N_eff': None},
    #'theory': Cobaya's protected key of the input dictionary.
    # Cobaya needs to ask some minimum theoretical requirements to a Boltzman Solver
    # (UC): you can choose between CAMB or CLASS
    # In this DEMO, we use CAMB and specify some CAMB arguments
    # such as the number of massive neutrinos
    # and the dark energy model
    #
    # ATTENTION: If you have CAMB/CLASS already installed and 
    # you are not using the likelihood conda environment 
    # or option (2) in cell (3) (Cobaya modules), you can add an extra key called 'path' within the camb dictionary
    # to point to your already installed CAMB code
    'theory': {'camb': 
               {'stop_at_error': True, 
                'extra_args':{'num_massive_neutrinos': 1,
                              'dark_energy_model': 'ppf',
                              # 'AccuracyBoost': 2.0
                             }}},
    #'sampler': Cobaya's protected key of the input dictionary.
    # In this DEMO, we use the 'evaluate' sampler to make a single computation of the posterior distributions
    'sampler': {'evaluate': None},  
    # 'packages_path': Cobaya's protected key of the input dictionary.
    # This is the variable you need to update
    # if you are running Cobaya with cobaya_modules (option (2) above).
    # If you are using the conda likelihood environment or option (1),
    # please, keep the line below commented
    #
    #'packages_path': modules_path,
    #
    #'output': Cobaya's protected key of the input dictionary.
    # Where are the results going to be stored, in case that the sampler produce output files? 
    # For example: chains...
    # (UC): modify the path below within 'output' to choose a name and a directory for those files
    'output': 'chains/cobaya_benchmark',
    #'debug': Cobaya's protected key of the input dictionary.
    # (UC): how much information you want Cobaya to print? If debug: True, it prints every single detail
    # that is going on internally in Cobaya
    'debug': False,
    #'timing': Cobaya's protected key of the input dictionary.
    # (UC): if timing: True, Cobaya returns how much time it took it to make a computation of the posterior
    # and how much time take each of the modules to perform their tasks
    'timing': False,
    #'force': Cobaya's protected key of the input dictionary.
    # (UC): if 'force': True, Cobaya forces deleting the previous output files, if found, with the same name
    'force': True
    }

In [ ]:
#'likelihood': Cobaya's protected key of the input dictionary.
# In this DEMO, we load the Euclid-Likelihood as an external function, and name it 'Euclid'
info['likelihood'] = {'Euclid': 
                     {'external': EuclidLikelihood, # Likelihood Class to be read as external
                     # Note: everything down below will overwrite the information read
                     # in the config folder
                     #
                     # Select which observables want to use during the analysis
                     # by setting them to True or False
                     'observables_selection': {
                         'WL': {'WL': True, 'GCphot': True, 'GCspectro': False},
                         'GCphot': {'GCphot': True, 'GCspectro': False},
                         'GCspectro': {'GCspectro': True},
                         'CG': {'CG': False},
                         'add_phot_RSD': False,
                         'matrix_transform_phot' : False # 'BNT' or 'BNT-test'
                     },
                     #'data': This give specifications for the paths of the input data files
                     'data': { 
                        #'sample' specifies the first folder below the main data folder
                        'sample': 'ExternalBenchmark',
                        #'spectro' and 'photo' specify paths to data files.
                        'spectro': {
                            # GC Spectro root name should contain z{:s} string
                            # to enable iteration over bins
                            'root': 'cov_power_galaxies_dk0p004_z{:s}.fits',
                            'redshifts': ["1.", "1.2", "1.4", "1.65"],
                            'edges': [0.9, 1.1, 1.3, 1.5, 1.8],
                            'cov_is_num': False,
                            'cov_nsim': 3500,
                            'scale_cuts_fourier': 'GCspectro-FourierSpace.yaml',
                            'root_mixing_matrix': 'mm_FS230degCircle_m3_nosm_obsz_z0.9-1.1.fits',
                            'Fourier': True},
                        'photo': {
                            'redshifts': [0.2095, 0.489, 0.619, 0.7335, 0.8445, 0.9595, 1.087, 1.2395, 1.45, 2.038],
                            'ndens_GC': 'niTab-EP10-RB00.dat',
                            'ndens_WL': 'niTab-EP10-RB00.dat',
                            'luminosity_ratio': 'luminosity_ratio.dat',
                            'photo_data': 'standard',
                            # Photometric root names should contain z{:s} string
                            # to specify IA model
                            'root_GC': 'Cls_{:s}_PosPos.dat',
                            'root_WL': 'Cls_{:s}_ShearShear.dat',
                            'root_XC': 'Cls_{:s}_PosShear.dat',
                            'root_mixing_matrix': 'fs2_mms_10zbins_32ellbins.fits',
                            'IA_model': 'zNLA',
                            # Photometric covariances root names should contain z{:s} string
                            # to specify how the covariance was calculated
                            'cov_GC': 'CovMat-PosPos-{:s}-20Bins.npz',
                            'cov_WL': 'CovMat-ShearShear-{:s}-20Bins.npz',
                            'cov_3x2pt': 'CovMat-3x2pt-{:s}-20Bins.npz',
                            'cov_model': 'Gauss',
                            'cov_is_num': False,
                            'cov_nsim': 10000,
                            'Fourier': True}}, 
                     # Plot the selected observables matrx
                     'plot_observables_selection': False,  
                     # Bias model
                     'bias_model': 1,
                     # Magnification bias model
                     # 1 -> linear interpolation
                     # 2 -> constant within bins
                     'magbias_model': 2,
                     # Nonlinear flags
                     # With this, the user can specify which nonlinear model they want
                     # For the time-being the available options are: 
                     # NL_flag_phot_matter
                     # 0 -> linear-only
                     # 1 -> Takahashi
                     # 2 -> Mead2016 (includes baryon corrections)
                     # 3 -> Mead2020 (w/o baryon corrections)
                     # 4 -> Mead2020_feedback (includes baryon corrections)
                     # 5 -> EE2
                     # 6 -> Bacco (matter)
                     'NL_flag_phot_matter': 0,
                     # Baryonic feedback flag selector
                     'NL_flag_phot_baryon': 0,
                     # NL_flag_spectro
                     # 0 -> linear-only
                     # 1 -> EFT
                     'NL_flag_spectro': 0,
                     # Intrinsic alignment flag
                     #     0 -> NLA
                     #     1 -> TATT
                     'IA_flag': 0,
                     # IR-resummation method
                        # 'DST' -> Discrete sine transform
                        # 'EH' -> Eisenstein-Hu                      
                     'IR_resum': 'DST',
                     'solver': 'camb',
                     # k values for extr'aPolation of the matter power spectrum and size k-array
                     'k_max_extrap': 500.0,
                     'k_min_extrap': 0.0001,   
                     'k_samp': 1000,
                     # z limit values and size z-array
                     'z_min': 0.0,
                     'z_max': 4.0,
                     'z_samp': 100,
                     # Baryonic feedback z_dependence flag selector for BCemu and Bacco emulators
                     'Baryon_redshift_model': True,
                     # Photometric galaxy bias flag
                     #     0 -> linear only
                     #     1 -> Non-linear Fast-PT
                     'NL_flag_phot_bias': 0,
                     # Use MG gamma
                     'use_gamma_MG': True,
                      # Use Weyl power spectrum (workaround approach)
                     'use_Weyl': False,
                     # Redshift dependent purity correction
                     'f_out_z_dep': False,
                     # Print theory predictions
                     'print_theory': False,
                     # Include spectroscopic galaxy clustering redshift error
                     'GCsp_z_err': False,
                     'params': {
                                # Spectro probe' : linear galaxy bias
                                'b1_spectro_bin1' : 1.414,
                                'b1_spectro_bin2' : 1.775,
                                'b1_spectro_bin3' : 2.042,
                                'b1_spectro_bin4' : 2.500,
                                # Spectro probe' : quadratic galaxy bias
                                'b2_spectro_bin1' : 1.061,
                                'b2_spectro_bin2' : 1.433,
                                'b2_spectro_bin3' : 1.903,
                                'b2_spectro_bin4' : 2.970,
                                # Spectro probe' : nonlocal quadratic galaxy bias
                                'bG2_spectro_bin1' : 0.0,
                                'bG2_spectro_bin2' : 0.0,
                                'bG2_spectro_bin3' : 0.0,
                                'bG2_spectro_bin4' : 0.0,
                                # Spectro probe' : nonlocal cubic galaxy bias
                                'bG3_spectro_bin1' : 0.0,
                                'bG3_spectro_bin2' : 0.0,
                                'bG3_spectro_bin3' : 0.0,
                                'bG3_spectro_bin4' : 0.0,
                                # Spectro probe' : EFT counter terms
                                'c0_spectro_bin1' : 6.490,
                                'c0_spectro_bin2' : 4.398,
                                'c0_spectro_bin3' : 3.323,
                                'c0_spectro_bin4' : 2.618,
                                'c2_spectro_bin1' : 37.801,
                                'c2_spectro_bin2' : 44.531,
                                'c2_spectro_bin3' : 43.376,
                                'c2_spectro_bin4' : 63.832,
                                'c4_spectro_bin1' : 56.729,
                                'c4_spectro_bin2' : 57.041,
                                'c4_spectro_bin3' : 45.242,
                                'c4_spectro_bin4' : 41.217,
                                'ck4_spectro_bin1' : 0.0,
                                'ck4_spectro_bin2' : 0.0,
                                'ck4_spectro_bin3' : 0.0,
                                'ck4_spectro_bin4' : 0.0,
                                # Spectro probe' : 1/bar(n)
                                'Psn_spectro_bin1' : 1627.758,
                                'Psn_spectro_bin2' : 3231.92,
                                'Psn_spectro_bin3' : 5680.44,
                                'Psn_spectro_bin4' : 10608.98,
                                # Spectro probe' : corretion to Poissonian shot noise
                                'aP_spectro_bin1' : 0.053,
                                'aP_spectro_bin2' : 0.139,
                                'aP_spectro_bin3' : 0.151,
                                'aP_spectro_bin4' : 0.311,
                                # Spectro probe' : scale-dependent shot noise
                                'e0k2_spectro_bin1' : 0.0,
                                'e0k2_spectro_bin2' : 0.0,
                                'e0k2_spectro_bin3' : 0.0,
                                'e0k2_spectro_bin4' : 0.0,
                                # Spectro probe' : scale- and mu-dependent shot noise
                                'e2k2_spectro_bin1' : 0.0,
                                'e2k2_spectro_bin2' : 0.0,
                                'e2k2_spectro_bin3' : 0.0,
                                'e2k2_spectro_bin4' : 0.0,
                                # Spectro probe' : unique purity nuisance parameter
                                'f_out' : 0.0,
                                # Spectro probe' : purity nuisance parameter per redshift bin
                                'f_out_1' : 0.195,
                                'f_out_2' : 0.204,
                                'f_out_3' : 0.306,
                                'f_out_4' : 0.121,
                                # Photometric probe' : IA
                                'a1_ia' : 1.72,
                                'eta1_ia' : -0.41,
                                'beta1_ia' : 2.17,
                                # Photometric probe' : polynomial galaxy bias
                                'b1_0_poly_photo' : 1.33291,
                                'b1_1_poly_photo' : -0.72414,
                                'b1_2_poly_photo' : 1.0183,
                                'b1_3_poly_photo' : -0.14913,
                                # Photometric probe' : polynomial magnification bias
                                'b0_mag_poly' : -1.50685,
                                'b1_mag_poly' : 1.35034,
                                'b2_mag_poly' : 0.08321,
                                'b3_mag_poly' : 0.04279,
                                # Photometric probe' : galaxy bias per redshift bin
                                'b1_photo_bin1' : 1.06533,
                                'b1_photo_bin2' : 1.20449,
                                'b1_photo_bin3' : 1.20498,
                                'b1_photo_bin4' : 1.31719,
                                'b1_photo_bin5' : 1.26084,
                                'b1_photo_bin6' : 1.32578,
                                'b1_photo_bin7' : 1.50557,
                                'b1_photo_bin8' : 1.475,
                                'b1_photo_bin9' : 1.45391,
                                'b1_photo_bin10' : 1.52119,
                                'b1_photo_bin11' : 1.63291,
                                'b1_photo_bin12' : 2.01904,
                                'b1_photo_bin13' : 2.61045,
                                # Photometric probe' : shift in redshift bins
                                'dz_1_WL' : -0.025749,
                                'dz_2_WL' : 0.022716,
                                'dz_3_WL' : -0.026032,
                                'dz_4_WL' : 0.012594,
                                'dz_5_WL' : 0.019285,
                                'dz_6_WL' : 0.008326,
                                'dz_7_WL' : 0.038207,
                                'dz_8_WL' : 0.002732,
                                'dz_9_WL' : 0.034066,
                                'dz_10_WL' : 0.049479,
                                'dz_11_WL' : 0.06649,
                                'dz_12_WL' : 0.000815,
                                'dz_13_WL' : 0.04907,
                                'dz_1_GCphot' : -0.025749,
                                'dz_2_GCphot' : 0.022716,
                                'dz_3_GCphot' : -0.026032,
                                'dz_4_GCphot' : 0.012594,
                                'dz_5_GCphot' : 0.019285,
                                'dz_6_GCphot' : 0.008326,
                                'dz_7_GCphot' : 0.038207,
                                'dz_8_GCphot' : 0.002732,
                                'dz_9_GCphot' : 0.034066,
                                'dz_10_GCphot' : 0.049479,
                                'dz_11_GCphot' : 0.06649,
                                'dz_12_GCphot' : 0.000815,
                                'dz_13_GCphot' : 0.04907,
                                # Theory' : gamma parametrization for Modified Gravity (MG)
                                'gamma_MG' : 0.55,
                                # Photometric probe' : magnification bias per redshift bin
                                'magnification_bias_1' : -0.90815,
                                'magnification_bias_2' : -0.9429000000000001,
                                'magnification_bias_3' : -0.99905,
                                'magnification_bias_4' : -0.8146499999999999,
                                'magnification_bias_5' : -0.6452,
                                'magnification_bias_6' : -0.77325,
                                'magnification_bias_7' : -0.5683,
                                'magnification_bias_8' : -0.2133999999999998,
                                'magnification_bias_9' : -0.015200000000000102,
                                'magnification_bias_10' : -0.04039999999999999,
                                'magnification_bias_11' : 0.6398999999999999,
                                'magnification_bias_12' : 0.91465,
                                'magnification_bias_13' : 1.6853500000000001,
                                # Photometric probe' : multiplicative bias parameters per redshift bin
                                'multiplicative_bias_1' : 0.0,
                                'multiplicative_bias_2' : 0.0,
                                'multiplicative_bias_3' : 0.0,
                                'multiplicative_bias_4' : 0.0,
                                'multiplicative_bias_5' : 0.0,
                                'multiplicative_bias_6' : 0.0,
                                'multiplicative_bias_7' : 0.0,
                                'multiplicative_bias_8' : 0.0,
                                'multiplicative_bias_9' : 0.0,
                                'multiplicative_bias_10' : 0.0,
                                'multiplicative_bias_11' : 0.0,
                                'multiplicative_bias_12' : 0.0,
                                'multiplicative_bias_13' : 0.0,
                                # Baryon parameters
                                # Bacco
                                'M1_z0_cen_bacco_bin1' : 11.340025775544985,
                                'M1_z0_cen_bacco_bin10' : 11.360811343412026,
                                'M1_z0_cen_bacco_bin11' : 11.360811343412026,
                                'M1_z0_cen_bacco_bin12' : 11.360811343412026,
                                'M1_z0_cen_bacco_bin13' : 11.360811343412026,
                                'M1_z0_cen_bacco_bin2' : 11.30961906881156,
                                'M1_z0_cen_bacco_bin3' : 11.366002025818862,
                                'M1_z0_cen_bacco_bin4' : 11.423530231429538,
                                'M1_z0_cen_bacco_bin5' : 11.479266623369345,
                                'M1_z0_cen_bacco_bin6' : 11.529947744472405,
                                'M1_z0_cen_bacco_bin7' : 11.569052829475833,
                                'M1_z0_cen_bacco_bin8' : 11.578833725581436,
                                'M1_z0_cen_bacco_bin9' : 11.496367669453079,
                                'M_c_bacco_bin1' : 14.158079090897681,
                                'M_c_bacco_bin10' : 15.179943239223197,
                                'M_c_bacco_bin11' : 15.179943239223197,
                                'M_c_bacco_bin12' : 15.179943239223197,
                                'M_c_bacco_bin13' : 15.179943239223197,
                                'M_c_bacco_bin2' : 14.413536974582364,
                                'M_c_bacco_bin3' : 14.513655381066439,
                                'M_c_bacco_bin4' : 14.59501140993621,
                                'M_c_bacco_bin5' : 14.667602932620955,
                                'M_c_bacco_bin6' : 14.736117785720888,
                                'M_c_bacco_bin7' : 14.803902874910724,
                                'M_c_bacco_bin8' : 14.873354165369339,
                                'M_c_bacco_bin9' : 14.947678648592403,
                                'M_inn_bacco_bin1' : 12.472,
                                'M_inn_bacco_bin10' : 8.119132619254136,
                                'M_inn_bacco_bin11' : 8.119132619254136,
                                'M_inn_bacco_bin12' : 8.119132619254136,
                                'M_inn_bacco_bin13' : 8.119132619254136,
                                'M_inn_bacco_bin2' : 11.098133175636903,
                                'M_inn_bacco_bin3' : 10.685545014634902,
                                'M_inn_bacco_bin4' : 10.379401082211185,
                                'M_inn_bacco_bin5' : 10.124549632511826,
                                'M_inn_bacco_bin6' : 9.89485323082124,
                                'M_inn_bacco_bin7' : 9.669674533440968,
                                'M_inn_bacco_bin8' : 9.423522683043274,
                                'M_inn_bacco_bin9' : 9.086255671995564,
                                'beta_bacco_bin1' : -0.1401597529600361,
                                'beta_bacco_bin10' : -0.1180328507054961,
                                'beta_bacco_bin11' : -0.1180328507054961,
                                'beta_bacco_bin12' : -0.1180328507054961,
                                'beta_bacco_bin13' : -0.1180328507054961,
                                'beta_bacco_bin2' : -0.14602731261184787,
                                'beta_bacco_bin3' : -0.15945179021822595,
                                'beta_bacco_bin4' : -0.1717165402260191,
                                'beta_bacco_bin5' : -0.18260340867450087,
                                'beta_bacco_bin6' : -0.1914516575772147,
                                'beta_bacco_bin7' : -0.19665359534532956,
                                'beta_bacco_bin8' : -0.1938633002915493,
                                'beta_bacco_bin9' : -0.16799956045034847,
                                'eta_bacco_bin1' : -0.34397323435383403,
                                'eta_bacco_bin10' : -0.3012527587604943,
                                'eta_bacco_bin11' : -0.3012527587604943,
                                'eta_bacco_bin12' : -0.3012527587604943,
                                'eta_bacco_bin13' : -0.3012527587604943,
                                'eta_bacco_bin2' : -0.3823001659654948,
                                'eta_bacco_bin3' : -0.3666759403458338,
                                'eta_bacco_bin4' : -0.34766555599217897,
                                'eta_bacco_bin5' : -0.32723937222645033,
                                'eta_bacco_bin6' : -0.3066851485202732,
                                'eta_bacco_bin7' : -0.28793191687619096,
                                'eta_bacco_bin8' : -0.2761849439738153,
                                'eta_bacco_bin9' : -0.29044476508897765,
                                'theta_inn_bacco_bin1' : -0.9790217925155928,
                                'theta_inn_bacco_bin10' : -0.5312787510657424,
                                'theta_inn_bacco_bin11' : -0.5312787510657424,
                                'theta_inn_bacco_bin12' : -0.5312787510657424,
                                'theta_inn_bacco_bin13' : -0.5312787510657424,
                                'theta_inn_bacco_bin2' : -1.3854647301685985,
                                'theta_inn_bacco_bin3' : -1.3604452515853098,
                                'theta_inn_bacco_bin4' : -1.2949667258085438,
                                'theta_inn_bacco_bin5' : -1.2056425079533284,
                                'theta_inn_bacco_bin6' : -1.0991845011332575,
                                'theta_inn_bacco_bin7' : -0.9805620838281162,
                                'theta_inn_bacco_bin8' : -0.8624918408856748,
                                'theta_inn_bacco_bin9' : -0.7968588249755786,
                                'theta_out_bacco_bin1' : 0.2672123790126238,
                                'theta_out_bacco_bin10' : 0.3569278707998519,
                                'theta_out_bacco_bin11' : 0.3569278707998519,
                                'theta_out_bacco_bin12' : 0.3569278707998519,
                                'theta_out_bacco_bin13' : 0.3569278707998519,
                                'theta_out_bacco_bin2' : 0.21845633941784046,
                                'theta_out_bacco_bin3' : 0.22229391886311567,
                                'theta_out_bacco_bin4' : 0.23124624733764712,
                                'theta_out_bacco_bin5' : 0.24338032280596764,
                                'theta_out_bacco_bin6' : 0.25802109222979985,
                                'theta_out_bacco_bin7' : 0.27483978056155306,
                                'theta_out_bacco_bin8' : 0.29287229590732955,
                                'theta_out_bacco_bin9' : 0.3076030069491944,
                                # BCemu parameters
                                'delta_bcemu_bin1' : 6.407442578598766,
                                'delta_bcemu_bin10' : 7.994110032082859,
                                'delta_bcemu_bin11' : 7.994110032082859,
                                'delta_bcemu_bin12' : 7.994110032082859,
                                'delta_bcemu_bin13' : 7.994110032082859,
                                'delta_bcemu_bin2' : 7.0846518716943025,
                                'delta_bcemu_bin3' : 7.226700228338714,
                                'delta_bcemu_bin4' : 7.343194626402527,
                                'delta_bcemu_bin5' : 7.470370166898234,
                                'delta_bcemu_bin6' : 7.638743359373677,
                                'delta_bcemu_bin7' : 7.891612514615537,
                                'delta_bcemu_bin8' : 8.252389998618296,
                                'delta_bcemu_bin9' : 8.702184867164187,
                                'deta_bcemu_bin1' : 0.08189803795251066,
                                'deta_bcemu_bin10' : 0.14550946518064195,
                                'deta_bcemu_bin11' : 0.14550946518064195,
                                'deta_bcemu_bin12' : 0.14550946518064195,
                                'deta_bcemu_bin13' : 0.14550946518064195,
                                'deta_bcemu_bin2' : 0.05028436917683151,
                                'deta_bcemu_bin3' : 0.04842000359519622,
                                'deta_bcemu_bin4' : 0.04869423933122641,
                                'deta_bcemu_bin5' : 0.04959595282843388,
                                'deta_bcemu_bin6' : 0.05011878848954406,
                                'deta_bcemu_bin7' : 0.04907477885822071,
                                'deta_bcemu_bin8' : 0.04681178740273659,
                                'deta_bcemu_bin9' : 0.048062862071344824,
                                'eta_bcemu_bin1' : 0.20680675958346154,
                                'eta_bcemu_bin10' : 0.044931761090432315,
                                'eta_bcemu_bin11' : 0.044931761090432315,
                                'eta_bcemu_bin12' : 0.044931761090432315,
                                'eta_bcemu_bin13' : 0.044931761090432315,
                                'eta_bcemu_bin2' : 0.22813237994091348,
                                'eta_bcemu_bin3' : 0.21989900653826788,
                                'eta_bcemu_bin4' : 0.2090499814223423,
                                'eta_bcemu_bin5' : 0.19645554090162448,
                                'eta_bcemu_bin6' : 0.18238365514915927,
                                'eta_bcemu_bin7' : 0.16686344766458094,
                                'eta_bcemu_bin8' : 0.14883503959461816,
                                'eta_bcemu_bin9' : 0.1240089652119986,
                                'gamma_bcemu_bin1' : 2.732738146480909,
                                'gamma_bcemu_bin10' : 3.1454520255604006,
                                'gamma_bcemu_bin11' : 3.1454520255604006,
                                'gamma_bcemu_bin12' : 3.1454520255604006,
                                'gamma_bcemu_bin13' : 3.1454520255604006,
                                'gamma_bcemu_bin2' : 2.794658925091356,
                                'gamma_bcemu_bin3' : 3.008141783989797,
                                'gamma_bcemu_bin4' : 3.2186226058208787,
                                'gamma_bcemu_bin5' : 3.425034377127418,
                                'gamma_bcemu_bin6' : 3.6237277892257937,
                                'gamma_bcemu_bin7' : 3.805233587893605,
                                'gamma_bcemu_bin8' : 3.952453083293017,
                                'gamma_bcemu_bin9' : 4.012632091622541,
                                'log10Mc_bcemu_bin1' : 13.402342835406102,
                                'log10Mc_bcemu_bin10' : 11.123969034839764,
                                'log10Mc_bcemu_bin11' : 11.123969034839764,
                                'log10Mc_bcemu_bin12' : 11.123969034839764,
                                'log10Mc_bcemu_bin13' : 11.123969034839764,
                                'log10Mc_bcemu_bin2' : 13.255603572838046,
                                'log10Mc_bcemu_bin3' : 13.39534479993441,
                                'log10Mc_bcemu_bin4' : 13.491189798802624,
                                'log10Mc_bcemu_bin5' : 13.517099731729203,
                                'log10Mc_bcemu_bin6' : 13.431948448938623,
                                'log10Mc_bcemu_bin7' : 13.159142065726643,
                                'log10Mc_bcemu_bin8' : 12.630530198678242,
                                'log10Mc_bcemu_bin9' : 11.768928839012311,
                                'mu_bcemu_bin1' : 1.081169575215787,
                                'mu_bcemu_bin10' : -0.006953332782079408,
                                'mu_bcemu_bin11' : -0.006953332782079408,
                                'mu_bcemu_bin12' : -0.006953332782079408,
                                'mu_bcemu_bin13' : -0.006953332782079408,
                                'mu_bcemu_bin2' : 1.0748796706532235,
                                'mu_bcemu_bin3' : 1.257505674386812,
                                'mu_bcemu_bin4' : 1.3953104854720346,
                                'mu_bcemu_bin5' : 1.4707848100185459,
                                'mu_bcemu_bin6' : 1.451326233210004,
                                'mu_bcemu_bin7' : 1.2738670043792995,
                                'mu_bcemu_bin8' : 0.8851923901330802,
                                'mu_bcemu_bin9' : 0.23572246186875243,
                                'thej_bcemu_bin1' : 3.6711008267835776,
                                'thej_bcemu_bin10' : 4.567495847826163,
                                'thej_bcemu_bin11' : 4.567495847826163,
                                'thej_bcemu_bin12' : 4.567495847826163,
                                'thej_bcemu_bin13' : 4.567495847826163,
                                'thej_bcemu_bin2' : 4.0853297524846175,
                                'thej_bcemu_bin3' : 4.1001870114090835,
                                'thej_bcemu_bin4' : 4.085897535238355,
                                'thej_bcemu_bin5' : 4.062128106370927,
                                'thej_bcemu_bin6' : 4.0416849190138695,
                                'thej_bcemu_bin7' : 4.0405617133891685,
                                'thej_bcemu_bin8' : 4.073797196972992,
                                'thej_bcemu_bin9' : 4.168188042786498,
                                # Redshift power law model
                                # Bacco
                                'M1_z0_cen_bacco_0' : 10.5,
                                'M_c_bacco_0' : 13.25,
                                'M_inn_bacco_0' : 13.4,
                                'beta_bacco_0' : -0.22,
                                'eta_bacco_0' : -0.3,
                                'theta_inn_bacco_0' : -0.86,
                                'theta_out_bacco_0' : 0.25,
                                'nu_M1_z0_cen_bacco' : 0.0,
                                'nu_M_c_bacco' : -0.15,
                                'nu_M_inn_bacco' : 0.0,
                                'nu_beta_bacco' : 0.0,
                                'nu_eta_bacco' : 0.0,
                                'nu_theta_inn_bacco' : 0.0,
                                'nu_theta_out_bacco' : 0.0,
                                # BCemu
                                'delta_bcemu_0' : 7.0,
                                'deta_bcemu_0' : 0.2,
                                'eta_bcemu_0' : 0.2,
                                'gamma_bcemu_0' : 2.5,
                                'log10Mc_bcemu_0' : 13.32,
                                'mu_bcemu_0' : 1.0,
                                'thej_bcemu_0' : 3.5,
                                'nu_delta_bcemu' : 0.0,
                                'nu_deta_bcemu' : 0.6,
                                'nu_eta_bcemu' : 0.0,
                                'nu_gamma_bcemu' : 0.0,
                                'nu_log10Mc_bcemu' : -0.15,
                                'nu_mu_bcemu' : 0.0,
                                'nu_thej_bcemu' : 0.0,
                                # Pivot redshift for calculation of fIA
                                'pivot_redshift' : 0.0,
                                # 1-point redshift error dispersion for GCspectro
                                'sigma_z' : 0.002,
                                # HMcode2020_feedback parameter (from arxiv:2009.01858)
                                'HMCode_logT_AGN' : 7.8,
                                # HMcode2016 parameters (values for matter-only from arxiv:1505.07833)
                                'HMCode_eta_baryon' : 0.603,
                                'HMCode_A_baryon' : 3.13,
                    }, 
                    }}

In [ ]:
from cloe.auxiliary.likelihood_yaml_handler import set_halofit_version
# Set the correct halofit version under the 'extra_args' field in ['theory']['camb']
set_halofit_version(info, info['likelihood']['Euclid']['NL_flag_phot_matter'], info['likelihood']['Euclid']['NL_flag_phot_baryon'])
# Now we are ready to run Cobaya

In [ ]:
# First: import model wrapper of Cobaya 
from cobaya.model import get_model
t1 = time.time()
# Second: create an instance of the `model` wrapper called model
model = get_model(info)
logposterior = model.logposterior({})
cobaya_loglike = logposterior.loglikes
chi2_cobaya = -2 * cobaya_loglike[0]

# Create an instance of the class EuclidLikelihood
like = EuclidLikelihood()
# Initialize default parameters for redshift, k-array, fiducial cosmology...
like.initialize()
# Get the cosmo_dictionary where all the cosmology + theory parameters are saved
like.passing_requirements(model, info, **model.provider.params)
# Update the cosmology dictionary with interpolators + basic quantities such as
# P_gg, P_delta...
like.cosmo.update_cosmo_dic(like.cosmo.cosmo_dic['z_win'], 0.05)
print('Time for calculation of the likelihood (Cobaya Benckmark): ', time.time()-t1)
print('   derived params: %r' % dict(zip(list(model.parameterization.derived_params()), logposterior.derived)))

# CASE 1: Run `CosmoSIS` with `Cobaya` under-the-hood
**User-case**: Here, we run the test case whereby `CosmoSIS` essentially calls `Cobaya` under-the-hood to calculate the cosmological parameters and matter power spectra. This is the equivalent to running
```bash
cosmosis run_cosmosis_with_cobaya.ini
```
in command line, except through a Python notebook.

In [ ]:
# Import cosmosis
import cosmosis

In [ ]:
# Setup the relevant CLOE module, based on the cosmosis_interface.py script
%run -i '../cosmosis/cosmosis_with_cobaya_interface.py'

euclid_module = cosmosis.FunctionModule('euclid', setup, execute)
# Pass on parameters under the [euclid] section in the ini file as a dictionary
params = {
    'euclid': {
        'config_file': '../cosmosis/cobaya_config_for_cosmosis.yaml',
        'cobaya_output': 'chains/case_A/cobaya_output'
    }
}
euclid_module.setup(params)

In [ ]:
#Define fixed set of parameter values as a dictionary
values = {
    'cosmological_parameters': {
        'ombh2': 0.022,
        'omch2': 0.1202,
        'H0': 67.0,
        'tau': 0.0925,
        'mnu': 0.06,
        'nnu': 3.046,
        'As': 2.12605e-9,
        'ns': 0.96,
        'w': -1.0,
        'wa': 0.0,
        'omk': 0.0,
        # Spectro probe' : linear galaxy bias
        'b1_spectro_bin1' : 1.414,
        'b1_spectro_bin2' : 1.775,
        'b1_spectro_bin3' : 2.042,
        'b1_spectro_bin4' : 2.500,
        # Spectro probe' : quadratic galaxy bias
        'b2_spectro_bin1' : 1.061,
        'b2_spectro_bin2' : 1.433,
        'b2_spectro_bin3' : 1.903,
        'b2_spectro_bin4' : 2.970,
        # Spectro probe' : nonlocal quadratic galaxy bias
        'bG2_spectro_bin1' : 0.0,
        'bG2_spectro_bin2' : 0.0,
        'bG2_spectro_bin3' : 0.0,
        'bG2_spectro_bin4' : 0.0,
        # Spectro probe' : nonlocal cubic galaxy bias
        'bG3_spectro_bin1' : 0.0,
        'bG3_spectro_bin2' : 0.0,
        'bG3_spectro_bin3' : 0.0,
        'bG3_spectro_bin4' : 0.0,
        # Spectro probe' : EFT counter terms
        'c0_spectro_bin1' : 6.490,
        'c0_spectro_bin2' : 4.398,
        'c0_spectro_bin3' : 3.323,
        'c0_spectro_bin4' : 2.618,
        'c2_spectro_bin1' : 37.801,
        'c2_spectro_bin2' : 44.531,
        'c2_spectro_bin3' : 43.376,
        'c2_spectro_bin4' : 63.832,
        'c4_spectro_bin1' : 56.729,
        'c4_spectro_bin2' : 57.041,
        'c4_spectro_bin3' : 45.242,
        'c4_spectro_bin4' : 41.217,
        'ck4_spectro_bin1' : 0.0,
        'ck4_spectro_bin2' : 0.0,
        'ck4_spectro_bin3' : 0.0,
        'ck4_spectro_bin4' : 0.0,
        # Spectro probe' : 1/bar(n)
        'Psn_spectro_bin1' : 1627.758,
        'Psn_spectro_bin2' : 3231.92,
        'Psn_spectro_bin3' : 5680.44,
        'Psn_spectro_bin4' : 10608.98,
        # Spectro probe' : corretion to Poissonian shot noise
        'aP_spectro_bin1' : 0.053,
        'aP_spectro_bin2' : 0.139,
        'aP_spectro_bin3' : 0.151,
        'aP_spectro_bin4' : 0.311,
        # Spectro probe' : scale-dependent shot noise
        'e0k2_spectro_bin1' : 0.0,
        'e0k2_spectro_bin2' : 0.0,
        'e0k2_spectro_bin3' : 0.0,
        'e0k2_spectro_bin4' : 0.0,
        # Spectro probe' : scale- and mu-dependent shot noise
        'e2k2_spectro_bin1' : 0.0,
        'e2k2_spectro_bin2' : 0.0,
        'e2k2_spectro_bin3' : 0.0,
        'e2k2_spectro_bin4' : 0.0,
        # Spectro probe' : unique purity nuisance parameter
        'f_out' : 0.0,
        # Spectro probe' : purity nuisance parameter per redshift bin
        'f_out_1' : 0.195,
        'f_out_2' : 0.204,
        'f_out_3' : 0.306,
        'f_out_4' : 0.121,
        # Photometric probe' : IA
        'a1_ia' : 1.72,
        'eta1_ia' : -0.41,
        'beta1_ia' : 2.17,
        # Photometric probe' : polynomial galaxy bias
        'b1_0_poly_photo' : 1.33291,
        'b1_1_poly_photo' : -0.72414,
        'b1_2_poly_photo' : 1.0183,
        'b1_3_poly_photo' : -0.14913,
        # Photometric probe' : polynomial magnification bias
        'b0_mag_poly' : -1.50685,
        'b1_mag_poly' : 1.35034,
        'b2_mag_poly' : 0.08321,
        'b3_mag_poly' : 0.04279,
        # Photometric probe' : galaxy bias per redshift bin
        'b1_photo_bin1' : 1.06533,
        'b1_photo_bin2' : 1.20449,
        'b1_photo_bin3' : 1.20498,
        'b1_photo_bin4' : 1.31719,
        'b1_photo_bin5' : 1.26084,
        'b1_photo_bin6' : 1.32578,
        'b1_photo_bin7' : 1.50557,
        'b1_photo_bin8' : 1.475,
        'b1_photo_bin9' : 1.45391,
        'b1_photo_bin10' : 1.52119,
        'b1_photo_bin11' : 1.63291,
        'b1_photo_bin12' : 2.01904,
        'b1_photo_bin13' : 2.61045,
        # Photometric probe' : shift in redshift bins
        'dz_1_WL' : -0.025749,
        'dz_2_WL' : 0.022716,
        'dz_3_WL' : -0.026032,
        'dz_4_WL' : 0.012594,
        'dz_5_WL' : 0.019285,
        'dz_6_WL' : 0.008326,
        'dz_7_WL' : 0.038207,
        'dz_8_WL' : 0.002732,
        'dz_9_WL' : 0.034066,
        'dz_10_WL' : 0.049479,
        'dz_11_WL' : 0.06649,
        'dz_12_WL' : 0.000815,
        'dz_13_WL' : 0.04907,
        'dz_1_GCphot' : -0.025749,
        'dz_2_GCphot' : 0.022716,
        'dz_3_GCphot' : -0.026032,
        'dz_4_GCphot' : 0.012594,
        'dz_5_GCphot' : 0.019285,
        'dz_6_GCphot' : 0.008326,
        'dz_7_GCphot' : 0.038207,
        'dz_8_GCphot' : 0.002732,
        'dz_9_GCphot' : 0.034066,
        'dz_10_GCphot' : 0.049479,
        'dz_11_GCphot' : 0.06649,
        'dz_12_GCphot' : 0.000815,
        'dz_13_GCphot' : 0.04907,
        # Theory' : gamma parametrization for Modified Gravity (MG)
        'gamma_MG' : 0.55,
        # Photometric probe' : magnification bias per redshift bin
        'magnification_bias_1' : -0.90815,
        'magnification_bias_2' : -0.9429000000000001,
        'magnification_bias_3' : -0.99905,
        'magnification_bias_4' : -0.8146499999999999,
        'magnification_bias_5' : -0.6452,
        'magnification_bias_6' : -0.77325,
        'magnification_bias_7' : -0.5683,
        'magnification_bias_8' : -0.2133999999999998,
        'magnification_bias_9' : -0.015200000000000102,
        'magnification_bias_10' : -0.04039999999999999,
        'magnification_bias_11' : 0.6398999999999999,
        'magnification_bias_12' : 0.91465,
        'magnification_bias_13' : 1.6853500000000001,
        # Photometric probe' : multiplicative bias parameters per redshift bin
        'multiplicative_bias_1' : 0.0,
        'multiplicative_bias_2' : 0.0,
        'multiplicative_bias_3' : 0.0,
        'multiplicative_bias_4' : 0.0,
        'multiplicative_bias_5' : 0.0,
        'multiplicative_bias_6' : 0.0,
        'multiplicative_bias_7' : 0.0,
        'multiplicative_bias_8' : 0.0,
        'multiplicative_bias_9' : 0.0,
        'multiplicative_bias_10' : 0.0,
        'multiplicative_bias_11' : 0.0,
        'multiplicative_bias_12' : 0.0,
        'multiplicative_bias_13' : 0.0,
        # Baryon parameters
        # Bacco
        'M1_z0_cen_bacco_bin1' : 11.340025775544985,
        'M1_z0_cen_bacco_bin10' : 11.360811343412026,
        'M1_z0_cen_bacco_bin11' : 11.360811343412026,
        'M1_z0_cen_bacco_bin12' : 11.360811343412026,
        'M1_z0_cen_bacco_bin13' : 11.360811343412026,
        'M1_z0_cen_bacco_bin2' : 11.30961906881156,
        'M1_z0_cen_bacco_bin3' : 11.366002025818862,
        'M1_z0_cen_bacco_bin4' : 11.423530231429538,
        'M1_z0_cen_bacco_bin5' : 11.479266623369345,
        'M1_z0_cen_bacco_bin6' : 11.529947744472405,
        'M1_z0_cen_bacco_bin7' : 11.569052829475833,
        'M1_z0_cen_bacco_bin8' : 11.578833725581436,
        'M1_z0_cen_bacco_bin9' : 11.496367669453079,
        'M_c_bacco_bin1' : 14.158079090897681,
        'M_c_bacco_bin10' : 15.179943239223197,
        'M_c_bacco_bin11' : 15.179943239223197,
        'M_c_bacco_bin12' : 15.179943239223197,
        'M_c_bacco_bin13' : 15.179943239223197,
        'M_c_bacco_bin2' : 14.413536974582364,
        'M_c_bacco_bin3' : 14.513655381066439,
        'M_c_bacco_bin4' : 14.59501140993621,
        'M_c_bacco_bin5' : 14.667602932620955,
        'M_c_bacco_bin6' : 14.736117785720888,
        'M_c_bacco_bin7' : 14.803902874910724,
        'M_c_bacco_bin8' : 14.873354165369339,
        'M_c_bacco_bin9' : 14.947678648592403,
        'M_inn_bacco_bin1' : 12.472,
        'M_inn_bacco_bin10' : 8.119132619254136,
        'M_inn_bacco_bin11' : 8.119132619254136,
        'M_inn_bacco_bin12' : 8.119132619254136,
        'M_inn_bacco_bin13' : 8.119132619254136,
        'M_inn_bacco_bin2' : 11.098133175636903,
        'M_inn_bacco_bin3' : 10.685545014634902,
        'M_inn_bacco_bin4' : 10.379401082211185,
        'M_inn_bacco_bin5' : 10.124549632511826,
        'M_inn_bacco_bin6' : 9.89485323082124,
        'M_inn_bacco_bin7' : 9.669674533440968,
        'M_inn_bacco_bin8' : 9.423522683043274,
        'M_inn_bacco_bin9' : 9.086255671995564,
        'beta_bacco_bin1' : -0.1401597529600361,
        'beta_bacco_bin10' : -0.1180328507054961,
        'beta_bacco_bin11' : -0.1180328507054961,
        'beta_bacco_bin12' : -0.1180328507054961,
        'beta_bacco_bin13' : -0.1180328507054961,
        'beta_bacco_bin2' : -0.14602731261184787,
        'beta_bacco_bin3' : -0.15945179021822595,
        'beta_bacco_bin4' : -0.1717165402260191,
        'beta_bacco_bin5' : -0.18260340867450087,
        'beta_bacco_bin6' : -0.1914516575772147,
        'beta_bacco_bin7' : -0.19665359534532956,
        'beta_bacco_bin8' : -0.1938633002915493,
        'beta_bacco_bin9' : -0.16799956045034847,
        'eta_bacco_bin1' : -0.34397323435383403,
        'eta_bacco_bin10' : -0.3012527587604943,
        'eta_bacco_bin11' : -0.3012527587604943,
        'eta_bacco_bin12' : -0.3012527587604943,
        'eta_bacco_bin13' : -0.3012527587604943,
        'eta_bacco_bin2' : -0.3823001659654948,
        'eta_bacco_bin3' : -0.3666759403458338,
        'eta_bacco_bin4' : -0.34766555599217897,
        'eta_bacco_bin5' : -0.32723937222645033,
        'eta_bacco_bin6' : -0.3066851485202732,
        'eta_bacco_bin7' : -0.28793191687619096,
        'eta_bacco_bin8' : -0.2761849439738153,
        'eta_bacco_bin9' : -0.29044476508897765,
        'theta_inn_bacco_bin1' : -0.9790217925155928,
        'theta_inn_bacco_bin10' : -0.5312787510657424,
        'theta_inn_bacco_bin11' : -0.5312787510657424,
        'theta_inn_bacco_bin12' : -0.5312787510657424,
        'theta_inn_bacco_bin13' : -0.5312787510657424,
        'theta_inn_bacco_bin2' : -1.3854647301685985,
        'theta_inn_bacco_bin3' : -1.3604452515853098,
        'theta_inn_bacco_bin4' : -1.2949667258085438,
        'theta_inn_bacco_bin5' : -1.2056425079533284,
        'theta_inn_bacco_bin6' : -1.0991845011332575,
        'theta_inn_bacco_bin7' : -0.9805620838281162,
        'theta_inn_bacco_bin8' : -0.8624918408856748,
        'theta_inn_bacco_bin9' : -0.7968588249755786,
        'theta_out_bacco_bin1' : 0.2672123790126238,
        'theta_out_bacco_bin10' : 0.3569278707998519,
        'theta_out_bacco_bin11' : 0.3569278707998519,
        'theta_out_bacco_bin12' : 0.3569278707998519,
        'theta_out_bacco_bin13' : 0.3569278707998519,
        'theta_out_bacco_bin2' : 0.21845633941784046,
        'theta_out_bacco_bin3' : 0.22229391886311567,
        'theta_out_bacco_bin4' : 0.23124624733764712,
        'theta_out_bacco_bin5' : 0.24338032280596764,
        'theta_out_bacco_bin6' : 0.25802109222979985,
        'theta_out_bacco_bin7' : 0.27483978056155306,
        'theta_out_bacco_bin8' : 0.29287229590732955,
        'theta_out_bacco_bin9' : 0.3076030069491944,
        # BCemu parameters
        'delta_bcemu_bin1' : 6.407442578598766,
        'delta_bcemu_bin10' : 7.994110032082859,
        'delta_bcemu_bin11' : 7.994110032082859,
        'delta_bcemu_bin12' : 7.994110032082859,
        'delta_bcemu_bin13' : 7.994110032082859,
        'delta_bcemu_bin2' : 7.0846518716943025,
        'delta_bcemu_bin3' : 7.226700228338714,
        'delta_bcemu_bin4' : 7.343194626402527,
        'delta_bcemu_bin5' : 7.470370166898234,
        'delta_bcemu_bin6' : 7.638743359373677,
        'delta_bcemu_bin7' : 7.891612514615537,
        'delta_bcemu_bin8' : 8.252389998618296,
        'delta_bcemu_bin9' : 8.702184867164187,
        'deta_bcemu_bin1' : 0.08189803795251066,
        'deta_bcemu_bin10' : 0.14550946518064195,
        'deta_bcemu_bin11' : 0.14550946518064195,
        'deta_bcemu_bin12' : 0.14550946518064195,
        'deta_bcemu_bin13' : 0.14550946518064195,
        'deta_bcemu_bin2' : 0.05028436917683151,
        'deta_bcemu_bin3' : 0.04842000359519622,
        'deta_bcemu_bin4' : 0.04869423933122641,
        'deta_bcemu_bin5' : 0.04959595282843388,
        'deta_bcemu_bin6' : 0.05011878848954406,
        'deta_bcemu_bin7' : 0.04907477885822071,
        'deta_bcemu_bin8' : 0.04681178740273659,
        'deta_bcemu_bin9' : 0.048062862071344824,
        'eta_bcemu_bin1' : 0.20680675958346154,
        'eta_bcemu_bin10' : 0.044931761090432315,
        'eta_bcemu_bin11' : 0.044931761090432315,
        'eta_bcemu_bin12' : 0.044931761090432315,
        'eta_bcemu_bin13' : 0.044931761090432315,
        'eta_bcemu_bin2' : 0.22813237994091348,
        'eta_bcemu_bin3' : 0.21989900653826788,
        'eta_bcemu_bin4' : 0.2090499814223423,
        'eta_bcemu_bin5' : 0.19645554090162448,
        'eta_bcemu_bin6' : 0.18238365514915927,
        'eta_bcemu_bin7' : 0.16686344766458094,
        'eta_bcemu_bin8' : 0.14883503959461816,
        'eta_bcemu_bin9' : 0.1240089652119986,
        'gamma_bcemu_bin1' : 2.732738146480909,
        'gamma_bcemu_bin10' : 3.1454520255604006,
        'gamma_bcemu_bin11' : 3.1454520255604006,
        'gamma_bcemu_bin12' : 3.1454520255604006,
        'gamma_bcemu_bin13' : 3.1454520255604006,
        'gamma_bcemu_bin2' : 2.794658925091356,
        'gamma_bcemu_bin3' : 3.008141783989797,
        'gamma_bcemu_bin4' : 3.2186226058208787,
        'gamma_bcemu_bin5' : 3.425034377127418,
        'gamma_bcemu_bin6' : 3.6237277892257937,
        'gamma_bcemu_bin7' : 3.805233587893605,
        'gamma_bcemu_bin8' : 3.952453083293017,
        'gamma_bcemu_bin9' : 4.012632091622541,
        'log10Mc_bcemu_bin1' : 13.402342835406102,
        'log10Mc_bcemu_bin10' : 11.123969034839764,
        'log10Mc_bcemu_bin11' : 11.123969034839764,
        'log10Mc_bcemu_bin12' : 11.123969034839764,
        'log10Mc_bcemu_bin13' : 11.123969034839764,
        'log10Mc_bcemu_bin2' : 13.255603572838046,
        'log10Mc_bcemu_bin3' : 13.39534479993441,
        'log10Mc_bcemu_bin4' : 13.491189798802624,
        'log10Mc_bcemu_bin5' : 13.517099731729203,
        'log10Mc_bcemu_bin6' : 13.431948448938623,
        'log10Mc_bcemu_bin7' : 13.159142065726643,
        'log10Mc_bcemu_bin8' : 12.630530198678242,
        'log10Mc_bcemu_bin9' : 11.768928839012311,
        'mu_bcemu_bin1' : 1.081169575215787,
        'mu_bcemu_bin10' : -0.006953332782079408,
        'mu_bcemu_bin11' : -0.006953332782079408,
        'mu_bcemu_bin12' : -0.006953332782079408,
        'mu_bcemu_bin13' : -0.006953332782079408,
        'mu_bcemu_bin2' : 1.0748796706532235,
        'mu_bcemu_bin3' : 1.257505674386812,
        'mu_bcemu_bin4' : 1.3953104854720346,
        'mu_bcemu_bin5' : 1.4707848100185459,
        'mu_bcemu_bin6' : 1.451326233210004,
        'mu_bcemu_bin7' : 1.2738670043792995,
        'mu_bcemu_bin8' : 0.8851923901330802,
        'mu_bcemu_bin9' : 0.23572246186875243,
        'thej_bcemu_bin1' : 3.6711008267835776,
        'thej_bcemu_bin10' : 4.567495847826163,
        'thej_bcemu_bin11' : 4.567495847826163,
        'thej_bcemu_bin12' : 4.567495847826163,
        'thej_bcemu_bin13' : 4.567495847826163,
        'thej_bcemu_bin2' : 4.0853297524846175,
        'thej_bcemu_bin3' : 4.1001870114090835,
        'thej_bcemu_bin4' : 4.085897535238355,
        'thej_bcemu_bin5' : 4.062128106370927,
        'thej_bcemu_bin6' : 4.0416849190138695,
        'thej_bcemu_bin7' : 4.0405617133891685,
        'thej_bcemu_bin8' : 4.073797196972992,
        'thej_bcemu_bin9' : 4.168188042786498,
        # Redshift power law model
        # Bacco
        'M1_z0_cen_bacco_0' : 10.5,
        'M_c_bacco_0' : 13.25,
        'M_inn_bacco_0' : 13.4,
        'beta_bacco_0' : -0.22,
        'eta_bacco_0' : -0.3,
        'theta_inn_bacco_0' : -0.86,
        'theta_out_bacco_0' : 0.25,
        'nu_M1_z0_cen_bacco' : 0.0,
        'nu_M_c_bacco' : -0.15,
        'nu_M_inn_bacco' : 0.0,
        'nu_beta_bacco' : 0.0,
        'nu_eta_bacco' : 0.0,
        'nu_theta_inn_bacco' : 0.0,
        'nu_theta_out_bacco' : 0.0,
        # BCemu
        'delta_bcemu_0' : 7.0,
        'deta_bcemu_0' : 0.2,
        'eta_bcemu_0' : 0.2,
        'gamma_bcemu_0' : 2.5,
        'log10Mc_bcemu_0' : 13.32,
        'mu_bcemu_0' : 1.0,
        'thej_bcemu_0' : 3.5,
        'nu_delta_bcemu' : 0.0,
        'nu_deta_bcemu' : 0.6,
        'nu_eta_bcemu' : 0.0,
        'nu_gamma_bcemu' : 0.0,
        'nu_log10Mc_bcemu' : -0.15,
        'nu_mu_bcemu' : 0.0,
        'nu_thej_bcemu' : 0.0,
        # Pivot redshift for calculation of fIA
        'pivot_redshift' : 0.0,
        # 1-point redshift error dispersion for GCspectro
        'sigma_z' : 0.002,
        # HMcode2020_feedback parameter (from arxiv:2009.01858)
        'HMCode_logT_AGN' : 7.8,
        # HMcode2016 parameters (values for matter-only from arxiv:1505.07833)
        'HMCode_eta_baryon' : 0.603,
        'HMCode_A_baryon' : 3.13,
    }
}

In [ ]:
# Setup the CosmoSIS pipeline with the modules, their parameters and the values to be sampled
modules = [euclid_module]
pipe = cosmosis.LikelihoodPipeline(params, values=values, modules=modules)

In [ ]:
t2 = time.time()
# Get the likelihood value from the current set of parameters
cosmosis_like, extra = pipe.likelihood(params)
# Define the sampler. We will use the test sampler to evaulate the logposterior of a single point
sampler_params = {
    "test": {
        # save all output to a directory 
        'save_dir': 'chains/case_A/cosmosis_output'
    }
}
# Make a Test sampler
sampler = cosmosis.samplers.TestSampler(sampler_params, pipe)
# The config method sets up the sampler.  It assumes the pipeline has already been configured.
sampler.config()
# Execute the sampler
sampler.execute()

print('Time for calculation of the likelihood (Cosmosis Case A): ', time.time()-t2)

## Comparison of $\chi^2$ value

In [ ]:
# Finally, we compare the chi^2 value obtained with the Cobaya benchmark value
chi2_cosmosis = -2 * cosmosis_like
print('chi2 of Cobaya benchmark = ', chi2_cobaya)
print('chi2 of CosmoSIS pipeline Case A = ', chi2_cosmosis)
print('Fractional Difference = ', (chi2_cobaya-chi2_cosmosis)/chi2_cobaya)

# Case 2: Run `CosmoSIS` independent of `Cobaya`

**User-case**: We test the second pipeline where `CosmoSIS` is being called independently of `Cobaya`. This is the equivalent to running
```bash
cosmosis run_cosmosis.ini
```
in command line, except through a Python notebook.

In [ ]:
# Setup the CAMB module, based on the camb_interface.py script. 
%run -i '../cosmosis/camb_interface.py'
camb_module = cosmosis.FunctionModule('camb',setup, execute)

# Setup the relevant CLOE module, based on the cosmosis_interface.py script
%run -i '../cosmosis/cosmosis_interface.py'
euclid_module = cosmosis.FunctionModule('euclid', setup, execute)

In [ ]:
# Pass on parameters under the [camb] and [euclid] section in the ini file as a dictionary
params = {
    'camb': {
        'mode': 'power',
        'feedback': 0,
        'use_ppf_w': True,
        'power_spectra': 'delta_tot weyl delta_nonu',
        'cloe_config_file' : '../cosmosis/config_for_cosmosis.yaml'
        },
    'euclid': {
        'config_file': '../cosmosis/config_for_cosmosis.yaml'
        }
}
camb_module.setup(params)
euclid_module.setup(params)

In [ ]:
#Define fixed set of parameter values as a dictionary
values = {
    'cosmological_parameters': {
        'ombh2': 0.022,
        'omch2': 0.1202,
        'h0': 0.67,
        'tau': 0.0925,
        'mnu': 0.06,
        'nnu': 3.046,
        'A_s': 2.12605e-9,
        'n_s': 0.96,
        'w': -1.0,
        'wa': 0.0,
        'omega_k': 0.0,
        # Theory' : gamma parametrization for Modified Gravity (MG)
        'gamma_MG' : 0.55,
        },
    'cloe_parameters':{
       # Spectro probe' : linear galaxy bias
                    'b1_spectro_bin1' : 1.414,
                    'b1_spectro_bin2' : 1.775,
                    'b1_spectro_bin3' : 2.042,
                    'b1_spectro_bin4' : 2.500,
                    # Spectro probe' : quadratic galaxy bias
                    'b2_spectro_bin1' : 1.061,
                    'b2_spectro_bin2' : 1.433,
                    'b2_spectro_bin3' : 1.903,
                    'b2_spectro_bin4' : 2.970,
                    # Spectro probe' : nonlocal quadratic galaxy bias
                    'bG2_spectro_bin1' : 0.0,
                    'bG2_spectro_bin2' : 0.0,
                    'bG2_spectro_bin3' : 0.0,
                    'bG2_spectro_bin4' : 0.0,
                    # Spectro probe' : nonlocal cubic galaxy bias
                    'bG3_spectro_bin1' : 0.0,
                    'bG3_spectro_bin2' : 0.0,
                    'bG3_spectro_bin3' : 0.0,
                    'bG3_spectro_bin4' : 0.0,
                    # Spectro probe' : EFT counter terms
                    'c0_spectro_bin1' : 6.490,
                    'c0_spectro_bin2' : 4.398,
                    'c0_spectro_bin3' : 3.323,
                    'c0_spectro_bin4' : 2.618,
                    'c2_spectro_bin1' : 37.801,
                    'c2_spectro_bin2' : 44.531,
                    'c2_spectro_bin3' : 43.376,
                    'c2_spectro_bin4' : 63.832,
                    'c4_spectro_bin1' : 56.729,
                    'c4_spectro_bin2' : 57.041,
                    'c4_spectro_bin3' : 45.242,
                    'c4_spectro_bin4' : 41.217,
                    'ck4_spectro_bin1' : 0.0,
                    'ck4_spectro_bin2' : 0.0,
                    'ck4_spectro_bin3' : 0.0,
                    'ck4_spectro_bin4' : 0.0,
                    # Spectro probe' : 1/bar(n)
                    'Psn_spectro_bin1' : 1627.758,
                    'Psn_spectro_bin2' : 3231.92,
                    'Psn_spectro_bin3' : 5680.44,
                    'Psn_spectro_bin4' : 10608.98,
                    # Spectro probe' : corretion to Poissonian shot noise
                    'aP_spectro_bin1' : 0.053,
                    'aP_spectro_bin2' : 0.139,
                    'aP_spectro_bin3' : 0.151,
                    'aP_spectro_bin4' : 0.311,
                    # Spectro probe' : scale-dependent shot noise
                    'e0k2_spectro_bin1' : 0.0,
                    'e0k2_spectro_bin2' : 0.0,
                    'e0k2_spectro_bin3' : 0.0,
                    'e0k2_spectro_bin4' : 0.0,
                    # Spectro probe' : scale- and mu-dependent shot noise
                    'e2k2_spectro_bin1' : 0.0,
                    'e2k2_spectro_bin2' : 0.0,
                    'e2k2_spectro_bin3' : 0.0,
                    'e2k2_spectro_bin4' : 0.0,
                    # Spectro probe' : unique purity nuisance parameter
                    'f_out' : 0.0,
                    # Spectro probe' : purity nuisance parameter per redshift bin
                    'f_out_1' : 0.195,
                    'f_out_2' : 0.204,
                    'f_out_3' : 0.306,
                    'f_out_4' : 0.121,
                    # Photometric probe' : IA
                    'a1_ia' : 1.72,
                    'eta1_ia' : -0.41,
                    'b1_ia' : 2.17,
                    # Photometric probe' : polynomial galaxy bias
                    'b1_0_poly_photo' : 1.33291,
                    'b1_1_poly_photo' : -0.72414,
                    'b1_2_poly_photo' : 1.0183,
                    'b1_3_poly_photo' : -0.14913,
                    # Photometric probe' : polynomial magnification bias
                    'b0_mag_poly' : -1.50685,
                    'b1_mag_poly' : 1.35034,
                    'b2_mag_poly' : 0.08321,
                    'b3_mag_poly' : 0.04279,
                    # Photometric probe' : galaxy bias per redshift bin
                    'b1_photo_bin1' : 1.06533,
                    'b1_photo_bin2' : 1.20449,
                    'b1_photo_bin3' : 1.20498,
                    'b1_photo_bin4' : 1.31719,
                    'b1_photo_bin5' : 1.26084,
                    'b1_photo_bin6' : 1.32578,
                    'b1_photo_bin7' : 1.50557,
                    'b1_photo_bin8' : 1.475,
                    'b1_photo_bin9' : 1.45391,
                    'b1_photo_bin10' : 1.52119,
                    'b1_photo_bin11' : 1.63291,
                    'b1_photo_bin12' : 2.01904,
                    'b1_photo_bin13' : 2.61045,
                    # Photometric probe' : shift in redshift bins
                    'dz_1_WL' : -0.025749,
                    'dz_2_WL' : 0.022716,
                    'dz_3_WL' : -0.026032,
                    'dz_4_WL' : 0.012594,
                    'dz_5_WL' : 0.019285,
                    'dz_6_WL' : 0.008326,
                    'dz_7_WL' : 0.038207,
                    'dz_8_WL' : 0.002732,
                    'dz_9_WL' : 0.034066,
                    'dz_10_WL' : 0.049479,
                    'dz_11_WL' : 0.06649,
                    'dz_12_WL' : 0.000815,
                    'dz_13_WL' : 0.04907,
                    'dz_1_GCphot' : -0.025749,
                    'dz_2_GCphot' : 0.022716,
                    'dz_3_GCphot' : -0.026032,
                    'dz_4_GCphot' : 0.012594,
                    'dz_5_GCphot' : 0.019285,
                    'dz_6_GCphot' : 0.008326,
                    'dz_7_GCphot' : 0.038207,
                    'dz_8_GCphot' : 0.002732,
                    'dz_9_GCphot' : 0.034066,
                    'dz_10_GCphot' : 0.049479,
                    'dz_11_GCphot' : 0.06649,
                    'dz_12_GCphot' : 0.000815,
                    'dz_13_GCphot' : 0.04907,
                    # Photometric probe' : magnification bias per redshift bin
                    'magnification_bias_1' : -0.90815,
                    'magnification_bias_2' : -0.9429000000000001,
                    'magnification_bias_3' : -0.99905,
                    'magnification_bias_4' : -0.8146499999999999,
                    'magnification_bias_5' : -0.6452,
                    'magnification_bias_6' : -0.77325,
                    'magnification_bias_7' : -0.5683,
                    'magnification_bias_8' : -0.2133999999999998,
                    'magnification_bias_9' : -0.015200000000000102,
                    'magnification_bias_10' : -0.04039999999999999,
                    'magnification_bias_11' : 0.6398999999999999,
                    'magnification_bias_12' : 0.91465,
                    'magnification_bias_13' : 1.6853500000000001,
                    # Photometric probe' : multiplicative bias parameters per redshift bin
                    'multiplicative_bias_1' : 0.0,
                    'multiplicative_bias_2' : 0.0,
                    'multiplicative_bias_3' : 0.0,
                    'multiplicative_bias_4' : 0.0,
                    'multiplicative_bias_5' : 0.0,
                    'multiplicative_bias_6' : 0.0,
                    'multiplicative_bias_7' : 0.0,
                    'multiplicative_bias_8' : 0.0,
                    'multiplicative_bias_9' : 0.0,
                    'multiplicative_bias_10' : 0.0,
                    'multiplicative_bias_11' : 0.0,
                    'multiplicative_bias_12' : 0.0,
                    'multiplicative_bias_13' : 0.0,
                    # Baryon parameters
                    # Bacco
                    'M1_z0_cen_bacco_bin1' : 11.340025775544985,
                    'M1_z0_cen_bacco_bin10' : 11.360811343412026,
                    'M1_z0_cen_bacco_bin11' : 11.360811343412026,
                    'M1_z0_cen_bacco_bin12' : 11.360811343412026,
                    'M1_z0_cen_bacco_bin13' : 11.360811343412026,
                    'M1_z0_cen_bacco_bin2' : 11.30961906881156,
                    'M1_z0_cen_bacco_bin3' : 11.366002025818862,
                    'M1_z0_cen_bacco_bin4' : 11.423530231429538,
                    'M1_z0_cen_bacco_bin5' : 11.479266623369345,
                    'M1_z0_cen_bacco_bin6' : 11.529947744472405,
                    'M1_z0_cen_bacco_bin7' : 11.569052829475833,
                    'M1_z0_cen_bacco_bin8' : 11.578833725581436,
                    'M1_z0_cen_bacco_bin9' : 11.496367669453079,
                    'M_c_bacco_bin1' : 14.158079090897681,
                    'M_c_bacco_bin10' : 15.179943239223197,
                    'M_c_bacco_bin11' : 15.179943239223197,
                    'M_c_bacco_bin12' : 15.179943239223197,
                    'M_c_bacco_bin13' : 15.179943239223197,
                    'M_c_bacco_bin2' : 14.413536974582364,
                    'M_c_bacco_bin3' : 14.513655381066439,
                    'M_c_bacco_bin4' : 14.59501140993621,
                    'M_c_bacco_bin5' : 14.667602932620955,
                    'M_c_bacco_bin6' : 14.736117785720888,
                    'M_c_bacco_bin7' : 14.803902874910724,
                    'M_c_bacco_bin8' : 14.873354165369339,
                    'M_c_bacco_bin9' : 14.947678648592403,
                    'M_inn_bacco_bin1' : 12.472,
                    'M_inn_bacco_bin10' : 8.119132619254136,
                    'M_inn_bacco_bin11' : 8.119132619254136,
                    'M_inn_bacco_bin12' : 8.119132619254136,
                    'M_inn_bacco_bin13' : 8.119132619254136,
                    'M_inn_bacco_bin2' : 11.098133175636903,
                    'M_inn_bacco_bin3' : 10.685545014634902,
                    'M_inn_bacco_bin4' : 10.379401082211185,
                    'M_inn_bacco_bin5' : 10.124549632511826,
                    'M_inn_bacco_bin6' : 9.89485323082124,
                    'M_inn_bacco_bin7' : 9.669674533440968,
                    'M_inn_bacco_bin8' : 9.423522683043274,
                    'M_inn_bacco_bin9' : 9.086255671995564,
                    'beta_bacco_bin1' : -0.1401597529600361,
                    'beta_bacco_bin10' : -0.1180328507054961,
                    'beta_bacco_bin11' : -0.1180328507054961,
                    'beta_bacco_bin12' : -0.1180328507054961,
                    'beta_bacco_bin13' : -0.1180328507054961,
                    'beta_bacco_bin2' : -0.14602731261184787,
                    'beta_bacco_bin3' : -0.15945179021822595,
                    'beta_bacco_bin4' : -0.1717165402260191,
                    'beta_bacco_bin5' : -0.18260340867450087,
                    'beta_bacco_bin6' : -0.1914516575772147,
                    'beta_bacco_bin7' : -0.19665359534532956,
                    'beta_bacco_bin8' : -0.1938633002915493,
                    'beta_bacco_bin9' : -0.16799956045034847,
                    'eta_bacco_bin1' : -0.34397323435383403,
                    'eta_bacco_bin10' : -0.3012527587604943,
                    'eta_bacco_bin11' : -0.3012527587604943,
                    'eta_bacco_bin12' : -0.3012527587604943,
                    'eta_bacco_bin13' : -0.3012527587604943,
                    'eta_bacco_bin2' : -0.3823001659654948,
                    'eta_bacco_bin3' : -0.3666759403458338,
                    'eta_bacco_bin4' : -0.34766555599217897,
                    'eta_bacco_bin5' : -0.32723937222645033,
                    'eta_bacco_bin6' : -0.3066851485202732,
                    'eta_bacco_bin7' : -0.28793191687619096,
                    'eta_bacco_bin8' : -0.2761849439738153,
                    'eta_bacco_bin9' : -0.29044476508897765,
                    'theta_inn_bacco_bin1' : -0.9790217925155928,
                    'theta_inn_bacco_bin10' : -0.5312787510657424,
                    'theta_inn_bacco_bin11' : -0.5312787510657424,
                    'theta_inn_bacco_bin12' : -0.5312787510657424,
                    'theta_inn_bacco_bin13' : -0.5312787510657424,
                    'theta_inn_bacco_bin2' : -1.3854647301685985,
                    'theta_inn_bacco_bin3' : -1.3604452515853098,
                    'theta_inn_bacco_bin4' : -1.2949667258085438,
                    'theta_inn_bacco_bin5' : -1.2056425079533284,
                    'theta_inn_bacco_bin6' : -1.0991845011332575,
                    'theta_inn_bacco_bin7' : -0.9805620838281162,
                    'theta_inn_bacco_bin8' : -0.8624918408856748,
                    'theta_inn_bacco_bin9' : -0.7968588249755786,
                    'theta_out_bacco_bin1' : 0.2672123790126238,
                    'theta_out_bacco_bin10' : 0.3569278707998519,
                    'theta_out_bacco_bin11' : 0.3569278707998519,
                    'theta_out_bacco_bin12' : 0.3569278707998519,
                    'theta_out_bacco_bin13' : 0.3569278707998519,
                    'theta_out_bacco_bin2' : 0.21845633941784046,
                    'theta_out_bacco_bin3' : 0.22229391886311567,
                    'theta_out_bacco_bin4' : 0.23124624733764712,
                    'theta_out_bacco_bin5' : 0.24338032280596764,
                    'theta_out_bacco_bin6' : 0.25802109222979985,
                    'theta_out_bacco_bin7' : 0.27483978056155306,
                    'theta_out_bacco_bin8' : 0.29287229590732955,
                    'theta_out_bacco_bin9' : 0.3076030069491944,
                    # BCemu parameters
                    'delta_bcemu_bin1' : 6.407442578598766,
                    'delta_bcemu_bin10' : 7.994110032082859,
                    'delta_bcemu_bin11' : 7.994110032082859,
                    'delta_bcemu_bin12' : 7.994110032082859,
                    'delta_bcemu_bin13' : 7.994110032082859,
                    'delta_bcemu_bin2' : 7.0846518716943025,
                    'delta_bcemu_bin3' : 7.226700228338714,
                    'delta_bcemu_bin4' : 7.343194626402527,
                    'delta_bcemu_bin5' : 7.470370166898234,
                    'delta_bcemu_bin6' : 7.638743359373677,
                    'delta_bcemu_bin7' : 7.891612514615537,
                    'delta_bcemu_bin8' : 8.252389998618296,
                    'delta_bcemu_bin9' : 8.702184867164187,
                    'deta_bcemu_bin1' : 0.08189803795251066,
                    'deta_bcemu_bin10' : 0.14550946518064195,
                    'deta_bcemu_bin11' : 0.14550946518064195,
                    'deta_bcemu_bin12' : 0.14550946518064195,
                    'deta_bcemu_bin13' : 0.14550946518064195,
                    'deta_bcemu_bin2' : 0.05028436917683151,
                    'deta_bcemu_bin3' : 0.04842000359519622,
                    'deta_bcemu_bin4' : 0.04869423933122641,
                    'deta_bcemu_bin5' : 0.04959595282843388,
                    'deta_bcemu_bin6' : 0.05011878848954406,
                    'deta_bcemu_bin7' : 0.04907477885822071,
                    'deta_bcemu_bin8' : 0.04681178740273659,
                    'deta_bcemu_bin9' : 0.048062862071344824,
                    'eta_bcemu_bin1' : 0.20680675958346154,
                    'eta_bcemu_bin10' : 0.044931761090432315,
                    'eta_bcemu_bin11' : 0.044931761090432315,
                    'eta_bcemu_bin12' : 0.044931761090432315,
                    'eta_bcemu_bin13' : 0.044931761090432315,
                    'eta_bcemu_bin2' : 0.22813237994091348,
                    'eta_bcemu_bin3' : 0.21989900653826788,
                    'eta_bcemu_bin4' : 0.2090499814223423,
                    'eta_bcemu_bin5' : 0.19645554090162448,
                    'eta_bcemu_bin6' : 0.18238365514915927,
                    'eta_bcemu_bin7' : 0.16686344766458094,
                    'eta_bcemu_bin8' : 0.14883503959461816,
                    'eta_bcemu_bin9' : 0.1240089652119986,
                    'gamma_bcemu_bin1' : 2.732738146480909,
                    'gamma_bcemu_bin10' : 3.1454520255604006,
                    'gamma_bcemu_bin11' : 3.1454520255604006,
                    'gamma_bcemu_bin12' : 3.1454520255604006,
                    'gamma_bcemu_bin13' : 3.1454520255604006,
                    'gamma_bcemu_bin2' : 2.794658925091356,
                    'gamma_bcemu_bin3' : 3.008141783989797,
                    'gamma_bcemu_bin4' : 3.2186226058208787,
                    'gamma_bcemu_bin5' : 3.425034377127418,
                    'gamma_bcemu_bin6' : 3.6237277892257937,
                    'gamma_bcemu_bin7' : 3.805233587893605,
                    'gamma_bcemu_bin8' : 3.952453083293017,
                    'gamma_bcemu_bin9' : 4.012632091622541,
                    'log10Mc_bcemu_bin1' : 13.402342835406102,
                    'log10Mc_bcemu_bin10' : 11.123969034839764,
                    'log10Mc_bcemu_bin11' : 11.123969034839764,
                    'log10Mc_bcemu_bin12' : 11.123969034839764,
                    'log10Mc_bcemu_bin13' : 11.123969034839764,
                    'log10Mc_bcemu_bin2' : 13.255603572838046,
                    'log10Mc_bcemu_bin3' : 13.39534479993441,
                    'log10Mc_bcemu_bin4' : 13.491189798802624,
                    'log10Mc_bcemu_bin5' : 13.517099731729203,
                    'log10Mc_bcemu_bin6' : 13.431948448938623,
                    'log10Mc_bcemu_bin7' : 13.159142065726643,
                    'log10Mc_bcemu_bin8' : 12.630530198678242,
                    'log10Mc_bcemu_bin9' : 11.768928839012311,
                    'mu_bcemu_bin1' : 1.081169575215787,
                    'mu_bcemu_bin10' : -0.006953332782079408,
                    'mu_bcemu_bin11' : -0.006953332782079408,
                    'mu_bcemu_bin12' : -0.006953332782079408,
                    'mu_bcemu_bin13' : -0.006953332782079408,
                    'mu_bcemu_bin2' : 1.0748796706532235,
                    'mu_bcemu_bin3' : 1.257505674386812,
                    'mu_bcemu_bin4' : 1.3953104854720346,
                    'mu_bcemu_bin5' : 1.4707848100185459,
                    'mu_bcemu_bin6' : 1.451326233210004,
                    'mu_bcemu_bin7' : 1.2738670043792995,
                    'mu_bcemu_bin8' : 0.8851923901330802,
                    'mu_bcemu_bin9' : 0.23572246186875243,
                    'thej_bcemu_bin1' : 3.6711008267835776,
                    'thej_bcemu_bin10' : 4.567495847826163,
                    'thej_bcemu_bin11' : 4.567495847826163,
                    'thej_bcemu_bin12' : 4.567495847826163,
                    'thej_bcemu_bin13' : 4.567495847826163,
                    'thej_bcemu_bin2' : 4.0853297524846175,
                    'thej_bcemu_bin3' : 4.1001870114090835,
                    'thej_bcemu_bin4' : 4.085897535238355,
                    'thej_bcemu_bin5' : 4.062128106370927,
                    'thej_bcemu_bin6' : 4.0416849190138695,
                    'thej_bcemu_bin7' : 4.0405617133891685,
                    'thej_bcemu_bin8' : 4.073797196972992,
                    'thej_bcemu_bin9' : 4.168188042786498,
                    # Redshift power law model
                    # Bacco
                    'M1_z0_cen_bacco_0' : 10.5,
                    'M_c_bacco_0' : 13.25,
                    'M_inn_bacco_0' : 13.4,
                    'beta_bacco_0' : -0.22,
                    'eta_bacco_0' : -0.3,
                    'theta_inn_bacco_0' : -0.86,
                    'theta_out_bacco_0' : 0.25,
                    'nu_M1_z0_cen_bacco' : 0.0,
                    'nu_M_c_bacco' : -0.15,
                    'nu_M_inn_bacco' : 0.0,
                    'nu_beta_bacco' : 0.0,
                    'nu_eta_bacco' : 0.0,
                    'nu_theta_inn_bacco' : 0.0,
                    'nu_theta_out_bacco' : 0.0,
                    # BCemu
                    'delta_bcemu_0' : 7.0,
                    'deta_bcemu_0' : 0.2,
                    'eta_bcemu_0' : 0.2,
                    'gamma_bcemu_0' : 2.5,
                    'log10Mc_bcemu_0' : 13.32,
                    'mu_bcemu_0' : 1.0,
                    'thej_bcemu_0' : 3.5,
                    'nu_delta_bcemu' : 0.0,
                    'nu_deta_bcemu' : 0.6,
                    'nu_eta_bcemu' : 0.0,
                    'nu_gamma_bcemu' : 0.0,
                    'nu_log10Mc_bcemu' : -0.15,
                    'nu_mu_bcemu' : 0.0,
                    'nu_thej_bcemu' : 0.0,
                    # Pivot redshift for calculation of fIA
                    'pivot_redshift' : 0.0,
                    # 1-point redshift error dispersion for GCspectro
                    'sigma_z' : 0.002,
                    # HMcode2020_feedback parameter (from arxiv:2009.01858)
                    'HMCode_logT_AGN' : 7.8,
                    # HMcode2016 parameters (values for matter-only from arxiv:1505.07833)
                    'HMCode_eta_baryon' : 0.603,
                    'HMCode_A_baryon' : 3.13,
    }
}

In [ ]:
# Setup the CosmoSIS pipeline with the modules, their parameters and the values to be sampled
modules = [camb_module,euclid_module]
pipe = cosmosis.LikelihoodPipeline(params, values=values, modules=modules)

In [ ]:
t3 = time.time()
# Get the likelihood value from the current set of parameters
like_cosmosis, extra = pipe.likelihood(params)
# Define the sampler. We will use the test sampler to evaulate the logposterior of a single point
sampler_params = {
    "test": {
        # save all output to a directory 
        'save_dir': 'chains/case_B/cosmosis_output'
    }
}
# Make a Test sampler
sampler = cosmosis.samplers.TestSampler(sampler_params, pipe)
# The config method sets up the sampler.  It assumes the pipeline has already been configured.
sampler.config()
# Execute the sampler
sampler.execute()

print('Time for calculation of the likelihood (Cosmosis Case B): ', time.time()-t3)

## Comparison of $\chi^2$ value

In [ ]:
# Finally, we compare the chi^2 value obtained with the Cobaya benchmark value
chi2_cosmosis = -2 * like_cosmosis
print('chi2 of Cobaya benchmark = ', chi2_cobaya)
print('chi2 from CosmoSIS pipeline Case B = ', chi2_cosmosis)
print('Fractional Difference = ', (chi2_cobaya-chi2_cosmosis)/chi2_cobaya)

## Plot internal quantities and cosmological observables

In [ ]:
import seaborn as sns
import matplotlib
matplotlib.rcParams['text.usetex'] = False
sns.set_theme(style="ticks")
sns.set_palette(sns.color_palette("Paired"))
%matplotlib inline

plt.rc('xtick',labelsize=20)
plt.rc('ytick',labelsize=20)
plt.rc('font',size=20)
plt.rc('axes', titlesize=25)
plt.rc('axes', labelsize=20)
plt.rc('lines', linewidth=3)
plt.rc('lines', markersize=6)
plt.rc('legend', fontsize=20)

In [ ]:
# Let's plot the background quantities of cosmo_dic
# See http://pf-ist-likelihood.pages.euclid-sgs.uk/likelihood-implementation/likelihood.cosmo.cosmology.html
# for extra information

fig, axs = plt.subplots(2, 2, figsize=(13,10))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Background quantities')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/'
z = np.loadtxt(cosmosis_output_dir+'growth_parameters/z.txt')

axs[0, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['H_z_func'](like.cosmo.cosmo_dic['z_win']),
              label='Cobaya Benchmark')
axs[0, 0].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/h.txt'),':',c = 'k',label='CosmoSIS')
axs[0, 0].set_xlabel(r'$z$')
axs[0, 0].set_ylabel(r'$H(z)$ $[\rm km \times s^{-1} Mpc^{-1}]$')
axs[0, 0].legend(frameon=False)

axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['d_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$D_{\rm{A}}(z)$ $[\rm{Mpc}]$')
axs[0, 1].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/da.txt'),':',c = 'k')
axs[0, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['r_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$r(z)$')
axs[0, 1].plot(z, np.loadtxt(cosmosis_output_dir+'distances/d_c.txt'),':',c = 'k')
axs[0, 1].set_xlabel(r'$z$')
axs[0, 1].legend(frameon=False)

axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['sigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$\sigma_8(z)$')
axs[1, 0].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/sigma_8.txt'),':',c = 'k')
axs[1, 0].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['fsigma8_z_func'](like.cosmo.cosmo_dic['z_win']),
              label = r'$f\sigma_8(z)$')
axs[1, 0].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/fsigma_8.txt'),':',c = 'k')
axs[1, 0].set_xlabel(r'$z$')
axs[1, 0].legend(frameon=False)

axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], like.cosmo.cosmo_dic['D_z_k'],
              label = r'$D(z)$')
axs[1, 1].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/d_z.txt'),':',c = 'k')
axs[1, 1].plot(like.cosmo.cosmo_dic['z_win'], 
               like.cosmo.cosmo_dic['f_z'](like.cosmo.cosmo_dic['z_win']),
              label = r'$f(z) = f\sigma_8/\sigma_8$')
axs[1, 1].plot(z, np.loadtxt(cosmosis_output_dir+'growth_parameters/f_z.txt'),':',c = 'k')
axs[1, 1].set_xlabel(r'$z$')
axs[1, 1].legend(frameon=False)

plt.tight_layout()
plt.subplots_adjust(top=0.90)


In [ ]:
# Let's plot the matter power spectrum 
k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

# WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
import warnings
warnings.filterwarnings('ignore')

fig, axs = plt.subplots(2, 2, figsize=(18,15))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Linear Matter Power Spectra')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/matter_power_lin/'
ks = like.cosmo.cosmo_dic['k_win']
h = values['cosmological_parameters']['h0']

axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,0].set_xlabel(k_label)
axs[0,0].set_ylabel(pm_label)
axs[0,0].legend(frameon=False)

axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,1].set_xlabel(k_label)
axs[0,1].set_ylabel(pm_label)
axs[0,1].legend(frameon=False)

axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,0].set_xlabel(k_label)
axs[1,0].set_ylabel(pm_label)
axs[1,0].legend(frameon=False)

#------
axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_delta'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,1].set_xlabel(k_label)
axs[1,1].set_ylabel(pm_label)
axs[1,1].legend(frameon=False)

In [ ]:
if info['likelihood']['Euclid']['NL_flag_phot_matter'] != 0:
        # Let's plot the nonlinear matter power spectrum 
        k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
        pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

        # WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
        import warnings
        warnings.filterwarnings('ignore')

        fig, axs = plt.subplots(2, 2, figsize=(18,15))
        for ax in axs.flatten():
                ax.tick_params(axis='both',which='both',direction='in')
        fig.suptitle('Nonlinear Matter Power Spectra (flag=%s)' %info['likelihood']['Euclid']['NL_flag_phot_matter'])

        cosmosis_output_dir = 'chains/case_B/cosmosis_output/matter_power_nl/'
        ks = like.cosmo.cosmo_dic['k_win']
        h = values['cosmological_parameters']['h0']

        axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_halomodel_recipe'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
                label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
        axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
                label='CosmoSIS')
        axs[0,0].set_xlabel(k_label)
        axs[0,0].set_ylabel(pm_label)
        axs[0,0].legend(frameon=False)

        axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_halomodel_recipe'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
                label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
        axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
                label='CosmoSIS')
        axs[0,1].set_xlabel(k_label)
        axs[0,1].set_ylabel(pm_label)
        axs[0,1].legend(frameon=False)

        axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_halomodel_recipe'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
                label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
        axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
                label='CosmoSIS')
        axs[1,0].set_xlabel(k_label)
        axs[1,0].set_ylabel(pm_label)
        axs[1,0].legend(frameon=False)

        #------
        axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_halomodel_recipe'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
                label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
        axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
                label='CosmoSIS')
        axs[1,1].set_xlabel(k_label)
        axs[1,1].set_ylabel(pm_label)
        axs[1,1].legend(frameon=False)

In [ ]:
# Let's plot the matter power spectrum 
k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

# WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
import warnings
warnings.filterwarnings('ignore')

fig, axs = plt.subplots(2, 2, figsize=(18,15))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Linear Power Spectra (CDM + Baryons)')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/cdm_baryon_power_lin/'
ks = like.cosmo.cosmo_dic['k_win']
h = values['cosmological_parameters']['h0']

axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,0].set_xlabel(k_label)
axs[0,0].set_ylabel(pm_label)
axs[0,0].legend(frameon=False)

axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,1].set_xlabel(k_label)
axs[0,1].set_ylabel(pm_label)
axs[0,1].legend(frameon=False)

axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,0].set_xlabel(k_label)
axs[1,0].set_ylabel(pm_label)
axs[1,0].legend(frameon=False)

#------
axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_cb'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,1].set_xlabel(k_label)
axs[1,1].set_ylabel(pm_label)
axs[1,1].legend(frameon=False)

In [ ]:
# Let's plot the matter power spectrum 
k_label           = r'$k$ $[\rm{Mpc}^{-1}]$'
pm_label          = r'$P_{\delta\delta} \left[\rm{Mpc}^3\right]$'

# WARNING: AT THE MOMENT, THE LARGE SCALES (small k values) in the power spectra are not physical
import warnings
warnings.filterwarnings('ignore')

fig, axs = plt.subplots(2, 2, figsize=(18,15))
for ax in axs.flatten():
    ax.tick_params(axis='both',which='both',direction='in')
fig.suptitle('Linear Weyl Power Spectra')

cosmosis_output_dir = 'chains/case_B/cosmosis_output/weyl_curvature_power_lin/'
ks = like.cosmo.cosmo_dic['k_win']
h = values['cosmological_parameters']['h0']

axs[0,0].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][0], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][0]))
axs[0,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[0,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,0].set_xlabel(k_label)
axs[0,0].set_ylabel(pm_label)
axs[0,0].legend(frameon=False)

axs[0,1].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][25], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][25]))
axs[0,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[25,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[0,1].set_xlabel(k_label)
axs[0,1].set_ylabel(pm_label)
axs[0,1].legend(frameon=False)

axs[1,0].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][50], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][50]))
axs[1,0].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[50,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,0].set_xlabel(k_label)
axs[1,0].set_ylabel(pm_label)
axs[1,0].legend(frameon=False)

#------
axs[1,1].loglog(ks, like.cosmo.cosmo_dic['Pk_weyl'].P(like.cosmo.cosmo_dic['z_win'][-1], ks), 'y-', 
           label=r"$z = {:.2f}$ Benchmark".format(like.cosmo.cosmo_dic['z_win'][-1]))
axs[1,1].loglog(np.loadtxt(cosmosis_output_dir+'k_h.txt')*h, np.loadtxt(cosmosis_output_dir+'p_k.txt')[-1,:]/h**3, 'k:', 
           label='CosmoSIS')
axs[1,1].set_xlabel(k_label)
axs[1,1].set_ylabel(pm_label)
axs[1,1].legend(frameon=False)